## test random environment with openai gym

#### note: the rendered worked better on windows than on linux (my old computer). in linux, the render would 'flash' - would finish really fast. I had to add a time.sleep method in linux, but this is not necessary in windows

In [3]:
import gym
import random

In [4]:
env = gym.make('CartPole-v0')
states = env.observation_space.shape[0]
actions = env.action_space.n

In [5]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = random.choice([0,1])
        n_state, reward, done, info = env.step(action)
        score += reward
    print(f'Episode: {episode} Score: {score}')

Episode: 1 Score: 14.0
Episode: 2 Score: 12.0
Episode: 3 Score: 26.0
Episode: 4 Score: 11.0
Episode: 5 Score: 14.0
Episode: 6 Score: 49.0
Episode: 7 Score: 14.0
Episode: 8 Score: 9.0
Episode: 9 Score: 23.0
Episode: 10 Score: 21.0


## create a deep learning model with keras

In [6]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [7]:
def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape=(1,states)))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [8]:
model = build_model(states, actions)

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 4)                 0         
                                                                 
 dense (Dense)               (None, 24)                120       
                                                                 
 dense_1 (Dense)             (None, 24)                600       
                                                                 
 dense_2 (Dense)             (None, 2)                 50        
                                                                 
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


## build agent with keras-rl

In [10]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [11]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [12]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

AttributeError: 'Sequential' object has no attribute '_compile_time_distribution_strategy'

In [14]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

RuntimeError: Your tried to test your agent but it hasn't been compiled yet. Please call `compile()` before `test()`.

In [15]:
_ = dqn.test(env, nb_episodes=5, visualize=True, )

RuntimeError: Your tried to test your agent but it hasn't been compiled yet. Please call `compile()` before `test()`.

In [1]:
env.close()

NameError: name 'env' is not defined

In [ ]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)

In [ ]:
del model
del dqn
del env

## reloading agent from memory

In [ ]:
# dqn is deleted
dqn.test(env, nb_episodes=5, visualize=True)

In [12]:
# reinstantiate the model
env = gym.make('CartPole-v0')
actions = env.action_space.n
states = env.observation_space.shape[0]
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

C:\Users\amoel.geogy\AppData\Local\Continuum\anaconda3\envs\reinfenv\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [13]:
# reload the saved weights
dqn.load_weights('dqn_weights.h5f')

In [18]:
_ = dqn.test(env, nb_episodes=1, visualize=True)

Testing for 1 episodes ...
Episode 1: reward: 200.000, steps: 200


In [17]:
env.close()